In [21]:
import pandas as pd
import re
import numpy as np

import requests
import urllib.request
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

In [22]:
import warnings
warnings.filterwarnings('ignore')

## 동적 크롤링

### 건강 데이터

In [60]:
# 데이터 프레임 생성
df = pd.DataFrame(np.zeros((180, 6), dtype = np.int), columns=['category', 'title', 'text','url', 'views', 'recommendation'])

In [61]:
df.head(3)

,category,title,text,url,views,recommendation
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0


In [62]:
url = "https://www.kbgoldenlifex.com/senior/XAA72P05010.kb"

# Selenium으로 웹 브라우저를 실행
driver = webdriver.Chrome()
driver.get(url)
page_num = 1
i = 0
while page_num <=9:
    # 웹 페이지가 로드될 때까지 잠시 대기
    driver.implicitly_wait(20) # 15초간 대기

    # 현재 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    anchor_tags = soup.find("div", class_ = "list-wrap").find_all("a")
    for tag in anchor_tags:
        href = tag.get('href')
        num = tag.get('data-blts-serno')

        if href and num:
            # 링크를 클릭하여 글 내용 페이지로 이동
            url_in = "https://www.kbgoldenlifex.com/senior/XAA72P05020.kb?bltsDstcd=01&bltsSerno="+str(num)+"&ctntDstcd=01"
            driver.get(url_in)

            # 글 내용이 로드될 때까지 잠시 대기
            driver.implicitly_wait(10) # 10초간 대기

            # 글 내용 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
            content_html = driver.page_source
            content_soup = BeautifulSoup(content_html, 'html.parser')

            # 글 내용 추
            category = content_soup.find("div", class_ = "view-title").find("span").text
            title = content_soup.find("div", class_ = "view-title").find("h2").text
            views = int(str(content_soup.find("ul", class_ = "conts-util")).split("<span>")[-1].split("</span>")[0].replace(",",""))
            try:
                recommend_url = soup.find("div", "news-health-banner").find("a")['href']
            except:
                recommend_url = np.nan
            txt = ""
            tmp = ""

            for con in content_soup.find_all("p"):
                tmp = txt
                txt += con.text.strip() + ""
                if (con.text.strip() == "▶ KB골든라이프X 연관 기사 보기"):
                    txt = tmp
                    recommend_url = con.find("a")['href']
                    break
            txt = ".".join(txt.split(".")[:-1]).replace("\xa0", " ").replace("\n", " ")
            txt = re.sub(r'\s+', ' ', txt)
            df.iloc[i,0] = category
            df.iloc[i,1] = title
            df.iloc[i,2] = txt
            df.iloc[i,3] = url_in
            df.iloc[i,4] = views
            df.iloc[i,5] = recommend_url
            i+=1
            # 이전 페이지로 돌아가기
            driver.back()
    print(page_num, "페이지를 가져왔습니다", "i:", i)
    page_num+=1
    try:
        page_url = 'a[name="pgEvent"][data-page-no="' + str(page_num) + '"]'
        next_page = driver.find_element(By.CSS_SELECTOR, page_url)
        next_page.click()
    except:
        print(page_num, "페이지를 찾을 수 없습니다.")
        break
        
# Selenium 사용이 끝나면 웹 브라우저를 닫음
driver.quit()

1 페이지를 가져왔습니다 i: 20
2 페이지를 가져왔습니다 i: 40
3 페이지를 가져왔습니다 i: 60
4 페이지를 가져왔습니다 i: 80
5 페이지를 가져왔습니다 i: 100
6 페이지를 가져왔습니다 i: 120
7 페이지를 가져왔습니다 i: 140
8 페이지를 가져왔습니다 i: 160
9 페이지를 가져왔습니다 i: 180
10 페이지를 찾을 수 없습니다.


In [63]:
df

,category,title,text,url,views,recommendation
0,건강,알고 먹자 영양제! ③ 비오틴은 정말 탈모에 효과가 있을까?,"‘비타민B7’ ‘비타민H’로도 불리는 비오틴(Biotin)은 모발과 피부, 손톱을 ...",https://www.kbgoldenlifex.com/senior/XAA72P050...,151,NaN
1,건강,허옇게 일어난 각질이여 안녕~ 여름철 ‘발 미인’ 되는 법,"샌들의 계절, 시선이 머무는 곳은 단연 발뒤꿈치다. 피지선이 없어 다른 부위보다 건...",https://www.kbgoldenlifex.com/senior/XAA72P050...,187,NaN
2,건강,"내 몸의 오랜 통증, 혹시 잘못된 자세 때문?","한 5분 걷거나 서 있었는데, 허리가 뻐근하고 저렸던 경험이 있는가? 갑자기 엉덩이...",https://www.kbgoldenlifex.com/senior/XAA72P050...,165,NaN
3,건강,나이가 들면 체질도 변한다! 시니어 다이어트 성공비법,"젊었을 때는 많이 먹고 적게 움직여서 살이 쪘지만, 나이가 들면 호르몬 때문에 살이...",https://www.kbgoldenlifex.com/senior/XAA72P050...,355,NaN
4,건강,여름이면 더 심해지는 ‘체취’ 퇴치법,흔히 ‘암내’라고 부르는 체취(몸 냄새)는 여름에 특히 더 심해진다. 보통 타인의 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,152,NaN
...,...,...,...,...,...,...
175,건강,잘 먹는 게 면역,쌉쌀한 사포닌은 호흡기를 건강하게 한다인삼의 성분으로 유명한 사포닌은 면역력과 호흡...,https://www.kbgoldenlifex.com/senior/XAA72P050...,375,NaN
176,건강,당신이 남보다 낫다는 착각을 버려라,"자기 자신에 집착하면 자기 자신을 해칠 뿐이다인간은 오랫동안 결점, 실수, 실패, ...",https://www.kbgoldenlifex.com/senior/XAA72P050...,78,NaN
177,건강,숨은 근육 관리하기,척추와 어깨 높이의 관계척추측만은 뒤에서 잘 보인다. 좌우 옆구리 라인이 짝짝이면 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,575,NaN
178,건강,유해물질과 함께 살아가는 법,"전직 식약청 독성부장, 일상 속 유해물질에 대해 말하다양기화 박사는 국내 최초로 ‘...",https://www.kbgoldenlifex.com/senior/XAA72P050...,360,NaN


In [240]:
df.to_csv('KB골든라이프X_건강.csv', index = False, encoding = 'utf-8')

### 여가 데이터

In [52]:
# 데이터 프레임 생성
df = pd.DataFrame(np.zeros((183, 6), dtype = np.int), columns=['category', 'title', 'text','url', 'views', 'recommendation'])

In [53]:
df.head()

,category,title,text,url,views,recommendation
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [54]:
url = "https://www.kbgoldenlifex.com/senior/XAA72P05011.kb"

# Selenium으로 웹 브라우저를 실행
driver = webdriver.Chrome()
driver.get(url)
page_num = 1
i = 0
while page_num <=10:
    # 웹 페이지가 로드될 때까지 잠시 대기
    driver.implicitly_wait(23) # 23초간 대기

    # 현재 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    anchor_tags = soup.find("div", class_ = "list-wrap").find_all("a")
    for tag in anchor_tags:
        href = tag.get('href')
        num = tag.get('data-blts-serno')

        if href and num:
            # 링크를 클릭하여 글 내용 페이지로 이동
            url_in = "https://www.kbgoldenlifex.com/senior/XAA72P05020.kb?bltsDstcd=01&bltsSerno="+str(num)+"&ctntDstcd=01"
            driver.get(url_in)

            # 글 내용이 로드될 때까지 잠시 대기
            driver.implicitly_wait(13) # 10초간 대기

            # 글 내용 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
            content_html = driver.page_source
            content_soup = BeautifulSoup(content_html, 'html.parser')

            # 글 내용 추
            category = content_soup.find("div", class_ = "view-title").find("span").text
            title = content_soup.find("div", class_ = "view-title").find("h2").text
            views = int(str(content_soup.find("ul", class_ = "conts-util")).split("<span>")[-1].split("</span>")[0].replace(",",""))
            try:
                recommend_url = soup.find("div", "news-health-banner").find("a")['href']
            except:
                recommend_url = np.nan
            txt = ""
            tmp = ""

            for con in content_soup.find_all("p"):
                tmp = txt
                txt += con.text.strip() + ""
                if (con.text.strip() == "▶ KB골든라이프X 연관 기사 보기"):
                    txt = tmp
                    try:
                        recommend_url = con.find("a")['href']
                    except:
                        recommend_url = recommend_url
                    break
            txt = ".".join(txt.split(".")[:-1]).replace("\xa0", " ").replace("\n", " ")
            txt = re.sub(r'\s+', ' ', txt)
            df.iloc[i,0] = category
            df.iloc[i,1] = title
            df.iloc[i,2] = txt
            df.iloc[i,3] = url_in
            df.iloc[i,4] = views
            df.iloc[i,5] = recommend_url
            i+=1
            # 이전 페이지로 돌아가기
            driver.back()
    print(page_num, "페이지를 가져왔습니다", "i:", i)
    page_num+=1
    try:
        page_url = 'a[name="pgEvent"][data-page-no="' + str(page_num) + '"]'
        next_page = driver.find_element(By.CSS_SELECTOR, page_url)
        next_page.click()
    except:
        print(page_num, "페이지를 찾을 수 없습니다.")
        break
        
# Selenium 사용이 끝나면 웹 브라우저를 닫음
driver.quit()

1 페이지를 가져왔습니다 i: 20
2 페이지를 가져왔습니다 i: 40
3 페이지를 가져왔습니다 i: 60
4 페이지를 가져왔습니다 i: 80
5 페이지를 가져왔습니다 i: 100
6 페이지를 가져왔습니다 i: 120
7 페이지를 가져왔습니다 i: 140
8 페이지를 가져왔습니다 i: 160
9 페이지를 가져왔습니다 i: 180
10 페이지를 가져왔습니다 i: 183
11 페이지를 찾을 수 없습니다.


In [55]:
df.head(23)

,category,title,text,url,views,recommendation
0,여가,"키오스크, 두렵지 않아! 즐기며 배우는 키오스크 활용법",최저임금이 크게 뛰기 시작한 2018년을 기점으로 주목받기 시작한 키오스크(공공장소...,https://www.kbgoldenlifex.com/senior/XAA72P050...,40,NaN
1,여가,"싱그러운 여름꽃, 네 이름은 뭐니? 식물 찾기 만능 앱","산에 핀 야생화나 화단에 핀 꽃의 이름이 궁금한데, 알 방법이 없어 답답했던 적이 ...",https://www.kbgoldenlifex.com/senior/XAA72P050...,83,NaN
2,여가,구석구석 로컬 투어 ③ ‘낮밤’ 없이 즐거운 광양 여행,‘구석구석 로컬 투어’ 시리즈는 국내 구석구석 로컬 여행지의 멋과 맛을 큐레이션합니...,https://www.kbgoldenlifex.com/senior/XAA72P050...,209,NaN
3,여가,올여름 인생 사진은 요기요! 여름꽃 ‘수국’ 명소,완연한 봄을 알리는 꽃이 벚꽃이라면 여름을 대표하는 꽃은 ‘수국’이다. 수국의 개화...,https://www.kbgoldenlifex.com/senior/XAA72P050...,103,NaN
4,여가,원산지표기법 바로알기! ‘국내산’ 배추에 ‘중국산’ 양념 김치는 국내산? 중국산?,원산지표시제도는 소비자에게 정확한 원산지 정보를 제공하고자 상품마다 생산 국적을 표...,https://www.kbgoldenlifex.com/senior/XAA72P050...,83,NaN
5,여가,"자외선 차단제, ‘제대로’만 발라도 10살은 어려 보인다!","피부 노화의 80%는 자외선과 환경오염, 흡연 등과 같은 환경적 요인에 기인한다. ...",https://www.kbgoldenlifex.com/senior/XAA72P050...,333,NaN
6,여가,6월은 보랏빛 향기를 타고! 전국 라벤더 명소 3,라벤더는 5월 말경 꽃이 피기 시작해 6월 초에 만개하는 대표적인 여름꽃이다. 고로...,https://www.kbgoldenlifex.com/senior/XAA72P050...,655,NaN
7,여가,구석구석 로컬 투어 ② ‘진짜’ 완주 여행,‘구석구석 로컬 투어’ 시리즈는 국내 구석구석 로컬 여행지의 멋과 맛을 큐레이션합니...,https://www.kbgoldenlifex.com/senior/XAA72P050...,318,NaN
8,여가,여행사는 옛말! 이제는 AI와 여행하는 시대,"챗GPT의 열기가 뜨겁다. 등장 100일 만에 사용자 1억 명을 돌파했고, 마이크로...",https://www.kbgoldenlifex.com/senior/XAA72P050...,132,NaN
9,여가,"오월, 봄바람과 함께 찾아온 전시","오월, 봄바람과 함께 찾아온 전시어느새 이마를 간질이는 훈풍을 느끼며 슬몃 미소짓게...",https://www.kbgoldenlifex.com/senior/XAA72P050...,276,https://www.kbgoldenlifex.com/senior/XAA72P050...


In [59]:
df.to_csv('KB골든라이프X_여가.csv', index = False, encoding = 'utf-8')

### 트렌드 데이터

In [41]:
# 데이터 프레임 생성
df = pd.DataFrame(np.zeros((144, 6), dtype = np.int), columns=['category', 'title', 'text','url', 'views', 'recommendation'])

In [42]:
df.head()

,category,title,text,url,views,recommendation
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [43]:
url = "https://www.kbgoldenlifex.com/senior/XAA72P05012.kb"

# Selenium으로 웹 브라우저를 실행
driver = webdriver.Chrome()
driver.get(url)
page_num = 1
i = 0
while page_num <=8:
    # 웹 페이지가 로드될 때까지 잠시 대기
    driver.implicitly_wait(23) # 23초간 대기

    # 현재 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    anchor_tags = soup.find("div", class_ = "list-wrap").find_all("a")
    for tag in anchor_tags:
        href = tag.get('href')
        num = tag.get('data-blts-serno')

        if href and num:
            # 링크를 클릭하여 글 내용 페이지로 이동
            url_in = "https://www.kbgoldenlifex.com/senior/XAA72P05020.kb?bltsDstcd=01&bltsSerno="+str(num)+"&ctntDstcd=01"
            driver.get(url_in)

            # 글 내용이 로드될 때까지 잠시 대기
            driver.implicitly_wait(13) # 10초간 대기

            # 글 내용 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
            content_html = driver.page_source
            content_soup = BeautifulSoup(content_html, 'html.parser')

            # 글 내용 추
            category = content_soup.find("div", class_ = "view-title").find("span").text
            title = content_soup.find("div", class_ = "view-title").find("h2").text
            views = int(str(content_soup.find("ul", class_ = "conts-util")).split("<span>")[-1].split("</span>")[0].replace(",",""))
            try:
                recommend_url = soup.find("div", "news-health-banner").find("a")['href']
            except:
                recommend_url = np.nan
            txt = ""
            tmp = ""

            for con in content_soup.find_all("p"):
                tmp = txt
                txt += con.text.strip() + ""
                if (con.text.strip() == "▶ KB골든라이프X 연관 기사 보기"):
                    txt = tmp
                    try:
                        recommend_url = con.find("a")['href']
                    except:
                        recommend_url = recommend_url
                    break
            txt = ".".join(txt.split(".")[:-1]).replace("\xa0", " ").replace("\n", " ")
            txt = re.sub(r'\s+', ' ', txt)
            df.iloc[i,0] = category
            df.iloc[i,1] = title
            df.iloc[i,2] = txt
            df.iloc[i,3] = url_in
            df.iloc[i,4] = views
            df.iloc[i,5] = recommend_url
            i+=1
            # 이전 페이지로 돌아가기
            driver.back()
    print(page_num, "페이지를 가져왔습니다", "i:", i)
    page_num+=1
    try:
        page_url = 'a[name="pgEvent"][data-page-no="' + str(page_num) + '"]'
        next_page = driver.find_element(By.CSS_SELECTOR, page_url)
        next_page.click()
    except:
        print(page_num, "페이지를 찾을 수 없습니다.")
        break
        
# Selenium 사용이 끝나면 웹 브라우저를 닫음
driver.quit()

1 페이지를 가져왔습니다 i: 20
2 페이지를 가져왔습니다 i: 40
3 페이지를 가져왔습니다 i: 60
4 페이지를 가져왔습니다 i: 80
5 페이지를 가져왔습니다 i: 100
6 페이지를 가져왔습니다 i: 120
7 페이지를 가져왔습니다 i: 140
8 페이지를 가져왔습니다 i: 144
9 페이지를 찾을 수 없습니다.


In [45]:
df.head(23)

,category,title,text,url,views,recommendation
0,트렌드,"New 직업 시리즈 ⑨ 취미도 즐기고, 돈도 벌고! 시니어 바리스타",은퇴 후 직업을 찾을 때 가장 먼저 고려해야 할 것 중 하나는 ‘취미’다. 좋아서 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,48,NaN
1,트렌드,"가성비부터 럭셔리까지, 실버타운, 어디가 좋아요?","‘세 끼 식사 제공, 의료진과 운동 매니저 상주, 부속 한의원, 주 2회 청소, 수...",https://www.kbgoldenlifex.com/senior/XAA72P050...,400,NaN
2,트렌드,"전기세가 폭탄이 되지 않게, 걱정 없이 여름 나는 절전법",전기 소비량이 연중 최대가 되는 한여름이 돌아왔다. 더불어 전기요금 인상 소식도 시...,https://www.kbgoldenlifex.com/senior/XAA72P050...,1817,NaN
3,트렌드,내가 전생에 왕이었다면 어떤 왕이었을까? 킹BTI 테스트,,https://www.kbgoldenlifex.com/senior/XAA72P050...,111257,NaN
4,트렌드,이게 무슨 말이래? 요즘 애들 신조어 사전,"군싹, 알잘딱깔센? 분명 우리나라 말인데, 도통 무슨 말인지 알 수 없는 1020세...",https://www.kbgoldenlifex.com/senior/XAA72P050...,222,NaN
5,트렌드,"환경도 보호하고, 돈도 번다! 떠오르는 요즘 재테크, 쓰테크!",쓰레기로 돈을 버는 ‘쓰테크(쓰레기와 재테크의 합성어)’가 인기다. 폐지나 공병 얘...,https://www.kbgoldenlifex.com/senior/XAA72P050...,359,NaN
6,트렌드,New 직업 시리즈 ⑧ 실버인지놀이지도사,고령화가 가속화되면서 노인 케어를 위한 직업이 주목받고 있다. ‘실버인지놀이지도사’...,https://www.kbgoldenlifex.com/senior/XAA72P050...,292,NaN
7,트렌드,New 직업 시리즈 ⑦일 자체가 힐링! 숲해설가,일상의 고단함을 다독여주는 공간이자 면역력까지 높인다는 숲. 코로나19 이후 지친 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,1690,NaN
8,트렌드,도심 속 산책이 필요한 당신에게,빠르게 돌아가는 서울 한복판에서도 느긋함과 기다림의 미학으로 손짓하는 휴식 공간을 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,140,NaN
9,트렌드,‘K-빨간 맛’의 질주,"‘불닭볶음면’, ‘엽기떡볶이’, ‘신길동 매운 짬뽕’, ‘실비 김치’ 등등. 이름만...",https://www.kbgoldenlifex.com/senior/XAA72P050...,67,NaN


In [48]:
df.tail()

,category,title,text,url,views,recommendation
139,트렌드,여전히 멋진 호텔들이 있다,브라슈 호텔브라슈 호텔은 프랑스 호텔 그룹 이보크가 세계적 산업디자이너 필립 스탁과...,https://www.kbgoldenlifex.com/senior/XAA72P050...,112,NaN
140,트렌드,핸드크림 뭐 쓰세요?,얼굴보다 손 관리를 더 철저하게노화는 얼굴뿐 아니라 우리 몸 전체에서 동시다발적으로...,https://www.kbgoldenlifex.com/senior/XAA72P050...,832,NaN
141,트렌드,유산균은 피부에도 좋을까?,남들은 좋다는 화장품인데 왜 내가 바르면 뾰루지가 날까?어느 브랜드의 발효 성분 에...,https://www.kbgoldenlifex.com/senior/XAA72P050...,29,NaN
142,트렌드,서점이 달라지고 있다,서점은 여전하다말끔한 공간에 감각적인 표지의 책들이 놓여 있다. 건물 지하 공간에 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,545,NaN
143,트렌드,갖지 않고 즐기는 삶,"굳이 돈을 주고 사서 내 것으로 만들지 않아도, 소유하는 것만큼 즐거울 수 있다.요...",https://www.kbgoldenlifex.com/senior/XAA72P050...,52,NaN


In [50]:
# recommendation 없는 행들
df[~df.recommendation.isna()].shape

(89, 6)

In [51]:
df.to_csv('KB골든라이프X_트렌드.csv', index = False, encoding = 'utf-8')